WA school zip codes sourced from https://eds.ospi.k12.wa.us/DirectoryEDS.aspx

In [1]:
# import packages
import pandas as pd
import glob

In [2]:
# load school data with zip codes
zip_raw = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/Other Data/Washington_School_Directory_20250109.csv')
zip = zip_raw[['ZipCode', 'City', 'LEAName','SchoolName','GradeCategory']]

# just need the 5 digit zip code
zip['ZipCode'] = zip['ZipCode'].str.split('-').str[0]

# make all strings lowercase
zip['SchoolName_lower'] = zip['SchoolName'].str.lower()
zip['City_lower'] = zip['City'].str.lower()
zip['District_lower'] = zip['LEAName'].str.lower()
# zip.head()

/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_20238/967220189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip['ZipCode'] = zip['ZipCode'].str.split('-').str[0]
/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_20238/967220189.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip['SchoolName_lower'] = zip['SchoolName'].str.lower()
/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_20238/967220189.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [3]:
# load fafsa data
fafsa = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/FAFSA/FAFSA Graduation Data.csv')

# make all strings lowercase
fafsa['SchoolName_lower'] = fafsa['Name'].str.lower()
fafsa['City_lower'] = fafsa['City'].str.lower()
fafsa['District_lower'] = fafsa['DistrictName'].str.lower()

In [4]:
# Merge the dataframes on the lowercase names
merged_df = pd.merge(fafsa, zip, left_on=['SchoolName_lower','City','District_lower'], right_on=['SchoolName_lower','City','District_lower'], how='inner')

# filter to selected columns
fafsa_zip = merged_df[['SchoolYear','As of Date','ZipCode','FinalCohort','Graduate','Submitted','Completed']]
# fafsa_zip.head()

# aggregate the data
fafsa_zip_aggregated = fafsa_zip.groupby(['SchoolYear', 'As of Date', 'ZipCode']).sum().reset_index()
# fafsa_zip_aggregated.head()

In [5]:
fafsa_zip_aggregated.to_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/FAFSA/FAFSA_aggregated.csv')

In [6]:
# load kindergarten readiness data
kind = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/Kindergarten Readiness/Kindergarten.csv')

# Convert names to lowercase
kind['SchoolName_lower'] = kind['SchoolName'].str.lower()
kind['DistrictName_lower'] = kind['DistrictName'].str.lower()
# kind.head()

In [7]:
# Merge the dataframes on the lowercase names
merged_kind_zip = pd.merge(kind, zip, left_on=['SchoolName_lower', 'DistrictName_lower'], right_on=['SchoolName_lower', 'District_lower'], how='inner')

# select columns
kind_zip = merged_kind_zip[['schoolyear','ZipCode','Numerator','Denominator']]

# aggregate the data
kind_zip_aggregated = kind_zip.groupby(['schoolyear', 'ZipCode']).sum().reset_index()
# kind_zip_aggregated.head()

In [8]:
kind_zip_aggregated.to_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/Kindergarten Readiness/kind_ready_agg.csv')

In [9]:
# load ALICE data and select cols
alice_raw = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/ALICE/ALICE_zip.csv')
alice = alice_raw[['Year','Zip Code','Households','Poverty Households','ALICE Households']]

# aggregate the data... it should already be aggregated, but just in case
alice_aggregated = alice.groupby(['Year', 'Zip Code']).sum().reset_index()
# alice_aggregated.head()

In [10]:
alice_aggregated.to_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/ALICE/ALICE_agg.csv')

In [ ]:
# Ensure all ZipCode columns are of the same datatype before merging
fafsa_zip_aggregated['ZipCode'] = fafsa_zip_aggregated['ZipCode'].astype(str)
kind_zip_aggregated['ZipCode'] = kind_zip_aggregated['ZipCode'].astype(str)
alice_aggregated['Zip Code'] = alice_aggregated['Zip Code'].astype(str)

# Merge fafsa_zip_aggregated and kind_zip_aggregated on ZipCode
merged_fafsa_kind = pd.merge(fafsa_zip_aggregated, kind_zip_aggregated, left_on='ZipCode', right_on='ZipCode', how='outer')

# Merge the result with alice_aggregated on ZipCode
merged_all = pd.merge(merged_fafsa_kind, alice_aggregated, left_on='ZipCode', right_on='Zip Code', how='outer')

# Drop the duplicate Zip Code column
merged_all = merged_all.drop(columns=['Zip Code'])

# Display the merged dataframe
merged_all.head()

,SchoolYear,As of Date,ZipCode,FinalCohort,Graduate,Submitted,Completed,schoolyear,Numerator,Denominator,Year,Zip Code,Households,Poverty Households,ALICE Households
0,2017.0,2016-01-31,98001,499.0,425.0,95.0,85.0,2013-14,71.0,218.0,2021.0,98001,11820.0,793.0,2067.0
1,2017.0,2016-01-31,98001,499.0,425.0,95.0,85.0,2014-15,63.0,206.0,2021.0,98001,11820.0,793.0,2067.0
2,2017.0,2016-01-31,98001,499.0,425.0,95.0,85.0,2015-16,170.0,392.0,2021.0,98001,11820.0,793.0,2067.0
3,2017.0,2016-01-31,98001,499.0,425.0,95.0,85.0,2016-17,201.0,438.0,2021.0,98001,11820.0,793.0,2067.0
4,2017.0,2016-01-31,98001,499.0,425.0,95.0,85.0,2017-18,193.0,413.0,2021.0,98001,11820.0,793.0,2067.0


Need to also join on year, which means data types need to be changed. Also have to convert schoolyear
As of Date... is there more than one per schoolyear? If so, need to take the max as of date per school year, then ditch that col